# Claim Extraction Experiment

In [ ]:
import os
from google import genai
from google.genai import types
from src.utils.loader_googlefile import load_video_file_from_metadata, save_video_file_metadata

import time
os.environ["GEMINI_API_KEY"] = ""

In [4]:
client = genai.Client()

### Video Upload

In [13]:
## fake video downloaded
video_path = "../data/socialdf/socialdf_vids/vids/C96nZIGIb_v/2024-07-27_06-24-04_UTC.mp4"
with open(video_path, 'rb') as f:
    video_file = client.files.upload(
        file=f,
        config=types.UploadFileConfig(mime_type="video/mp4")
    )

print(f"Uploaded file: {video_file.name}")

# Wait for the file to be processed
print("Waiting for video processing...")
while video_file.state == "PROCESSING":
    time.sleep(2)
    video_file = client.files.get(name=video_file.name)

if video_file.state == "FAILED":
    raise ValueError("Video processing failed")

print("Video processing complete!")

Uploaded file: files/2umem6nzmg3o
Waiting for video processing...
Video processing complete!


In [15]:
prompt = """
    Por favor faça uma extração abrangente deste vídeo incluindo:
    - Principais tópicos discutidos
    - Participantes do vídeo e tom do discurso
    """
    
# Generate the summary
print("Generating summary...")
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        types.Part.from_uri(file_uri=video_file.uri, mime_type=video_file.mime_type),
        prompt
    ]
)


Generating summary...


In [18]:
print(response.text)

Aqui está uma extração abrangente do vídeo:

**Principais Tópicos Discutidos:**
*   **Jogo de Azar (Gambling):** O tema central é o jogo de azar, especificamente a pressão social e os avisos que Steve Harvey (e, por extensão, o espectador) recebe sobre o assunto.
*   **Percepções Sociais sobre o Jogo de Azar:** O vídeo explora a visão comum de que o jogo de azar é prejudicial, leva à perda de dinheiro e ao vício.
*   **Experiência Pessoal vs. Conselhos Comuns:** Steve Harvey contrasta os conselhos negativos que recebe com sua própria experiência, sugerindo que ele não vê o jogo de azar como algo inerentemente ruim.
*   **A Marca Rollbit:** A presença constante do logotipo "ROLLBIT" na parte inferior do vídeo sugere uma promoção ou patrocínio de uma plataforma de cripto-jogos/apostas.

**Pontos-Chave ou Destaques:**
*   Steve Harvey relata que as pessoas o aconselham a parar de jogar, alertando que ele "perderá todo o seu dinheiro e se tornará viciado".
*   Ele desconsidera esses avisos

## Prototyping with LangChain

In [ ]:
from langchain_core.runnables import RunnableLambda
import asyncio
import json
import pickle


In [9]:

def analyze_with_prompt(video_file, prompt_text):
    """
    Analyze video with a specific prompt
    
    Args:
        video_file: Uploaded video file object
        prompt_text: Analysis prompt
        
    Returns:
        Analysis result
    """
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=[
            types.Part.from_uri(
                file_uri=video_file.uri, 
                mime_type=video_file.mime_type
            ),
            prompt_text
        ]
    )
    return response.text

async def analyze_video_sequential(video_file, first_prompt, second_prompt_template):
    """
    Upload video, get first analysis, then use result in second analysis using LangChain
    
    Args:
        video_path: Path to the video file
        first_prompt: Prompt for initial analysis (e.g., summarization)
        second_prompt_template: Template for second prompt with {summary} placeholder
        
    Returns:
        Dictionary with both results
    """
    
    # Step 2: Create first runnable for summarization
    def run_first_analysis(input_data):
        print("\n--- Running First Analysis (Summarization) ---")
        result = analyze_with_prompt(video_file, first_prompt)
        return {"summary": result, "video_file": video_file}
    
    first_runnable = RunnableLambda(run_first_analysis)
    
    # Step 3: Create second runnable that uses the summary
    def run_second_analysis(input_data):
        print("\n--- Running Second Analysis (Using Summary) ---")
        summary = input_data["summary"]
        video_file = input_data["video_file"]
        
        # Format the second prompt with the summary
        formatted_prompt = second_prompt_template.format(summary=summary)
        
        result = analyze_with_prompt(video_file, formatted_prompt)
        return {"summary": summary, "claim": result}
    
    second_runnable = RunnableLambda(run_second_analysis)
    
    # Step 4: Chain them together
    chain = first_runnable | second_runnable
    
    # Step 5: Execute the chain
    results = await chain.ainvoke({})
    
    return results

# Synchronous wrapper
def analyze_video_sequential_sync(video_path, first_prompt, second_prompt_template):
    """
    Synchronous version of sequential video analysis
    """
    return asyncio.run(analyze_video_sequential(video_path, first_prompt, second_prompt_template))

In [14]:
video = load_video_file_from_metadata("video_metadata.json")


summarization_prompt = """
    Por favor faça uma extração abrangente deste vídeo incluindo:
    - Principais tópicos discutidos
    - Participantes do vídeo e tom do discurso
    """

claim_prompt = """
    Com base no seguinte resumo do vídeo, gere em uma sentença a mensagme do vídeo
    """

results = analyze_video_sequential_sync(
    video,
    summarization_prompt,
    claim_prompt
)

Loaded video file: files/2umem6nzmg3o


RuntimeError: asyncio.run() cannot be called from a running event loop

In [13]:
results

<coroutine object analyze_video_sequential at 0x7707c6b2bc40>